In [3]:
import os 
import sys

spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.4-src.zip'))
sys.path.insert(0, os.path.join(spark_home, 'python'))
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.0
      /_/

Using Python version 2.7.11 (default, Dec  6 2015 18:57:58)
SparkSession available as 'spark'.


In [4]:
DatasetRaw = sc.textFile('114_congress.csv')

def remove_header(itr_index, itr):
    return iter(list(itr)[1:]) if itr_index == 0 else itr

DatasetRaw = DatasetRaw.mapPartitionsWithIndex(remove_header).cache()

In [5]:
DatasetRaw = DatasetRaw.map(lambda line: [str(i + 1) + ":" + x + " " for i, x in enumerate(line.split(',')[3:])])

In [6]:
DatasetRaw = DatasetRaw.zipWithIndex().map(lambda line: [str(line[1]) + ' '] + line[0]).map(lambda line: (1,line + ['\n'])).map(lambda line: (line[0],''.join(line[1])))

In [9]:
NewDataset = DatasetRaw.reduceByKey(lambda v1, v2: v1+v2).collect()[0][1]

In [10]:
encoding = 'utf-8'
import io

with io.open('new_votes.txt', 'w', encoding=encoding) as file:
    file.write(NewDataset)

In [58]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import PCA, StandardScaler

#Kmeans
dataset = spark.read.format("libsvm").load("new_votes.txt")
kmeans = KMeans().setK(2).setSeed(1)
model = kmeans.fit(dataset)
centers = model.clusterCenters()
predictions = model.transform(dataset)


In [59]:
predlist = predictions.select('prediction').collect()
prediction_array = [int(i.prediction) for i in predlist]


In [60]:
#PCA
standardizer = StandardScaler(withMean=True, withStd=True, inputCol='features', outputCol='features_2')
model = standardizer.fit(dataset)
dataset = model.transform(dataset)
pca = PCA(k=2, inputCol='features_2', outputCol='pcaFeatures')
model = pca.fit(dataset)
result = model.transform(dataset).select('pcaFeatures')

# Putting all together
PCApredictions = model.transform(dataset)

In [67]:
PCAlist = PCApredictions.select('pcaFeatures').collect()
PCA_array = [[float(i.pcaFeatures[0]),float(i.pcaFeatures[1])] for i in PCAlist]


In [68]:
import numpy as np
PCA_array = np.array(PCA_array)
prediction_array = np.array(prediction_array)

In [69]:
import matplotlib.pyplot as plt

plt.scatter(x=PCA_array[:,0], y=PCA_array[:,1], c=prediction_array, s=100)
plt.show()

In [ ]:
sc.stop()